# Resumen

En esta práctica, aplicaremos los conocimientos adquiridos en las clases de teoría y prácticas anteriores para crear un índice invertido considerando restricciones impuestas por el hardware subyacente a la hora de construirlo. En concreto, las tareas a realizar en esta práctica son:

1. **Construir un índice usando el algoritmo *naïve* y ser capaz de guardarlo/cargarlo de disco**. Usaremos el mismo corpus de documentos que en la práctica anterior.


# Requisitos para la construcción del índice

In [1]:
# Añade aquí los imports que necesites
import sys
import pickle as pkl
import array
import os
import timeit
import contextlib
from pathlib import Path
from urllib.parse import urlparse
import urllib.request
import zipfile

## Límite de memoria disponible

Para comprobar que no rebasas los límites de uso de memoria pre-establecidos para realizar la construcción del índice de la manera adecuada, debes ejecutar este notebook lanzando `jupyter-notebook` desde un terminal del *shell*, en el que previamente hayas ejecutado la siguiente orden:

```bash
$ ulimit -v $(echo $(( 1024 * 1024 )))
```

In [2]:
!ulimit -a | grep "virtual memory"

virtual memory              (kbytes, -v) unlimited


Si has establecido correctamente los límites de uso de memoria del proceso `python3` que está ejecutando este *notebook*, el resultado de ejecutar la orden anterior debería ser:
```
virtual memory              (kbytes, -v) 1048576
```

## Corpus de documentos. Directorios de entrada y salida

Trabajaremos en esta práctica con el mismo corpus de documentos que en la anterior. Ejecuta la siguiente celda para obtenerlo en caso de que no lo tengas ya descargado en el mismo directorio del notebook.

In [3]:
# URL del corpus de documentos
data_url = 'http://ditec.um.es/~rtitos/docencia/ri/practica1-datos.zip'

local_filename = os.path.basename(urlparse(data_url).path)
urllib.request.urlretrieve(data_url, local_filename)
zip_ref = zipfile.ZipFile(local_filename, 'r')
zip_ref.extractall()
zip_ref.close()

A continuación se definen las rutas relativas a los directorios que utilizaremos en el notebook tanto para generar los ficheros resultantes del indizado, como los directorios en los que se ubican los ficheros de entrada (corpus, tests, etc.)

In [4]:
# Directorios de entrada y salida (generados)
corpus_dir = 'data/corpus'
toy_dir = 'data/toy'
out_dir_naive='output/naive/index'
out_dir_naive_toy='output/naive/toy_index'
out_dir_bsbi='output/bsbi/index'
out_dir_bsbi_toy='output/bsbi/toy_index'

# Cambia esto a "True" para indexar el corpus íntegro en memoria (requiere más de 1GiB de memoria)
build_naive_index_full = True

terms_filename = 'terms.dict'
docs_filename = 'docs.dict'
postings_index_filename = 'postings.index'
postings_metadata_filename = 'postings_metadata.dict'

Ejecuta la siguiente celda para asegurarte de que todo está en su sitio:

In [5]:
from pathlib import Path
import os

# Check if all directories exist
def check_directories(*directories):
    for path in directories:
        directory = Path(path)
        if not directory.is_dir():
            raise FileNotFoundError(f"Cannot find required directory at path '{path}'.")
        else:
            print(f"Found required directory at '{path}'.")

# Create output directories
def create_directories(*directories):
    for path in directories:
        try:
            os.makedirs(path, exist_ok=False)
            print(f"Output directory '{path}' has been created.")
        except FileExistsError:
            print(f"Output directory '{path}' already exists.")
            pass
        except Exception as e:
            print(f"Could not create directory at path '{path}'. Error: {e}")

check_directories(corpus_dir, toy_dir)
create_directories(out_dir_naive, out_dir_naive_toy, out_dir_bsbi, out_dir_bsbi_toy)


Found required directory at 'data/corpus'.
Found required directory at 'data/toy'.
Output directory 'output/naive/index' already exists.
Output directory 'output/naive/toy_index' already exists.
Output directory 'output/bsbi/index' already exists.
Output directory 'output/bsbi/toy_index' already exists.


El índice se almacenará en el directorio indicado por la variable `out_dir_naive`. También utilizaremos un directorio, dado por el valor de `out_dir_naive_toy`, donde se generará un índice de prueba a partir de datos de juguete. Por su parte, `tmp_dir` indica la ruta al directorio en el que se guardarán algunos archivos temporales para los índices de juguete.

## La clase auxiliar *IdMap*

Para hacer más eficiente la construcción de índices, representaremos los términos del vocabulario como *termIDs* (en lugar de cadenas), donde cada *termID* es un número de serie único. Podemos construir el mapeo de términos a termIDs sobre la marcha mientras procesamos la colección. Del mismo modo, también representamos los documentos como *docIDs* (en lugar de cadenas).

Con el fin de simplificar la correspondencia entre cadenas e ids numéricos, debes programar una clase auxiliar llamada `IdMap`. Esta clase se encargará de asignar biyectivamente término a termID y doc a docID.

Para esto, la clase deberá contener dos atributos:
* `str_to_id`: Un diccionario que mapeará cada cadena a su id numéricos, permitiendo un acceso eficiente al id numérico a partir de la cadena.
* `id_to_str`: Una lista para asociar cada id numérico (posición en la lista) a su correspondiente cadena de caracteres, permitiendo un acceso y almacenamiento eficientes de ids numéricos a cadenas.


A la vista de estos requisitos, programa las funciones `_get_str` y `_get_id` en el siguiente código. La única interfaz a esta clase es proporcionada por `__getitem__` que obtiene el mapeo correcto dependiendo del tipo de clave.

**Documentación recomendada**: *Si lo necesitas, consulta [la documentación sobre métodos especiales (o "mágicos")](https://docs.python.org/3.7/reference/datamodel.html#special-method-names) de Python, tal como como `__getitem__` ). También puede resultarte útil este [breve tutorial](https://www.omkarpathak.in/2018/04/11/python-getitem-and-setitem/).*

In [6]:
class IdMap:
    """Helper class to store a mapping from strings to ids."""
    def __init__(self):
        self.str_to_id = {}
        self.id_to_str = []
        
    def __len__(self):
        """Return number of terms stored in the IdMap"""
        ### Begin your code
        # Asegurarse de que ambas estructuras tienen la misma longitud
        assert len(self.str_to_id) == len(self.id_to_str), \
            "Inconsistencia: str_to_id y id_to_str tienen longitudes diferentes"
        return len(self.id_to_str)
        ### End your code
        
    def _get_str(self, i):
        """Returns the string corresponding to a given id (`i`)."""
        ### Begin your code
        # Verificar que el índice está en el rango válido
        if i < 0 or i >= len(self.id_to_str):
            raise IndexError(f"ID {i} fuera de rango. Rango válido: [0, {len(self.id_to_str)-1}]")
        return self.id_to_str[i]
        ### End your code
        
    def _get_id(self, s):
        """Returns the id corresponding to a string (`s`). 
        If `s` is not in the IdMap yet, then assigns a new id and returns the new id.
        """
        # idx is ID mapped to this string (or next unassigned ID if key is missing)
        ### Begin your code
        # Si la cadena ya existe en el diccionario, devuelve su ID
        if s in self.str_to_id:
            idx = self.str_to_id[s]
        else:
            # Si no existe, asigna un nuevo ID (el siguiente disponible)
            idx = len(self.id_to_str)
            # Añade el mapeo en ambas estructuras
            self.str_to_id[s] = idx
            self.id_to_str.append(s)
        ### End your code
        assert len(self.str_to_id) == len(self.id_to_str)  
        return idx
        
            
    def __getitem__(self, key):
        """If `key` is a integer, use _get_str; 
           If `key` is a string, use _get_id;"""
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        else:
            raise TypeError

Asegúrese de que supera los siguientes tests sencillos:

In [7]:
testIdMap = IdMap()
assert testIdMap['a'] == 0, "Unable to add a new string to the IdMap"
assert testIdMap['bcd'] == 1, "Unable to add a new string to the IdMap"
assert testIdMap['a'] == 0, "Unable to retrieve the id of an existing string"
assert testIdMap[1] == 'bcd', "Unable to retrive the string corresponding to a\
                                given id"
try:
    testIdMap[2]
except IndexError as e:
    assert True, "Doesn't throw an IndexError for out of range numeric ids"
assert len(testIdMap) == 2

# Construcción de un índice invertido en memoria: Algoritmo *naïve*

Ahora vamos a construir un índice invertido siguiendo un algoritmo *naïve* como el que se describe al comienzo del tema 2 de teoría, el cual no tiene en cuenta los requisitos de espacio en memoria del algoritmo ni las características del hardware del computador en el que se ejecuta. Según este algoritmo, el índice al completo, incluyendo tanto su diccionario de términos como las *postings lists* de cada término del vocabulario, se mantienen en memoria mientras se va construyendo el índice.

**IMPORTANTE: Llevar a cabo la construcción de esta forma resultará únicamente posible en el caso de que no estemos usando `ulimit` para imponer limitaciones el uso de memoria por parte del proceso encargado de ejecutar este *notebook* de Jupyter  (ver indicaciones al comienzo del cuaderno).**

Recordemos que, para construir un índice, los pasos generales son:
1. Escanear la colección reuniendo todos los pares término-documento.
2. Ordenar los pares con el término como clave primaria y el docID como clave secundaria.
3. Generar una lista con los docID de cada término

En nuestro caso, ya que no nos preocupará el tamaño de las estructuras de datos que mantengamos en memoria, realizaremos la construcción de una manera ligeramente distinta: en lugar de generar una lista de tuplas (termId,docId) durante el escaneo del corpus, utilizaremos un diccionario Python para mantener la correspondencia entre cada término (identificado por su *termId*) y la lista de documentos (*docIds*) en que aparece (*postings list*). El uso de estas estructuras de datos en memoria nos simplifica la tarea por varias razones:
- No es necesario ordenar por *termId* (orden primario de tuplas (termid, docid)), sino que los términos del diccionario están automáticamente ordenados por *termId* gracias a que, desde Python 3.7, los diccionarios son ordenados (mantienen el orden de inserción). Puesto que los términos se insertan en orden creciente de *termId*, a la hora de escribir el índice en disco podremos simplemente recorrer el diccionario a la hora de escribir las *postings lists* sin que sea necesario realizar ninguna ordenación previa.
- No es necesario ordenar por *docId* (orden secundario) sino que para cada término construiremos su *postings list* manteniendo el orden de los *docIds* en todo momento, al tiempo que evitamos duplicados. Esto es sencillo puesto que los documentos se escanean en orden creciente de *docId* (se generará un nuevo *docId* al considerar cada nuevo documento del corpus). Para cada término encontrado en el corpus, simplemente se comprueba si ya existe en el diccionario; en tal caso, basta con comparar el *docId* actual con el del último *posting* de la lista asociada el término, para decidir si resulta necesario añadirlo o no.

In [8]:
class NaiveIndex:
    """A class that implements a naive inverted index built entirely in memory
    
    Attributes
    ---------
    term_id_map(IdMap): For mapping terms to termIDs
    doc_id_map(IdMap): For mapping relative paths of documents (eg 
        0/3dradiology.stanford.edu_) to docIDs
    data_dir(str): Path to data
    out_dir(str): Path to directory where index files will be saved to/loaded from
    postings: Dictionary mapping: termID->postings lists [docid1,docid2...]
    postings_metadata: Dictionary mapping: termID->(start_position_in_index_file, 
                                                    number_of_postings_in_list,
                                                    length_in_bytes_of_postings_list)
    """
    def __init__(self, data_dir, out_dir):
        self.term_id_map = IdMap()
        self.doc_id_map = IdMap()
        self.data_dir = data_dir
        self.output_dir = out_dir
        self.postings = {}
        self.postings_metadata = {}

## *Parsear* los documentos del *corpus*

Programa los métodos `parse_all_subdirectories` y `parse_directory` de la clase `NaiveIndex`, de la forma que se indica:

- `parse_directory`: Escanea los documentos existentes en un directorio y los añade a un diccionario de *postings* que mantiene la asociación entre cada *termID* y su *postings list* (lista de *docIds*). Recibe como argumentos la ruta al directorio a escanear y el diccionario de *postings* sobre el que debe operar.

- `parse_all_subdirectories`: Debe hacer uso de la función anterior, para escanear todos los subdirectorios ('0', '1', etc.) en los que se dividen los ficheros del corpus. Como resultado de ejecutar esta función, el atributo `postings` de la clase `NaiveIndex` contendrá un diccionario con tantos elementos como términos distintos encontrados en el corpus, y para cada uno de ellos los *docId* en que aparece. 

**ACLARACIÓN**: *En las siguientes celdas, el hacer que `NaiveIndex` herede de `NaiveIndex` no es más que una forma sencilla de añadir un nuevo método a una clase ya existente. Aunque esto puede resultar confuso, en realidad se utiliza simplemente para dividir en varias celdas una definición de clase dentro de un cuaderno de Jupyter.*

In [9]:
class NaiveIndex(NaiveIndex):
    def parse_directory(self, dir_path, postings):
        """Parse all documents at the given path, inserts
        to the dictionary of postings lists given as argument.
        Should use self.term_id_map and self.doc_id_map to get termIDs and docIDs.
        """
        # read the lines of each doc, extract its terms and insert the docid in the list of postlist of that term
        ### Begin your code
        # Obtener lista de archivos en el directorio y ordenarlos
        filenames = sorted(os.listdir(dir_path))
        
        # Procesar cada archivo
        for filename in filenames:
            filepath = os.path.join(dir_path, filename)
            
            # Verificar que es un archivo (no un directorio)
            if os.path.isfile(filepath):
                # Obtener o asignar docID para este documento
                # Usar ruta relativa desde data_dir para consistencia
                relative_path = os.path.relpath(filepath, self.data_dir)
                doc_id = self.doc_id_map[relative_path]
                
                # Leer el contenido del archivo
                with open(filepath, 'r', encoding='utf-8') as f:
                    content = f.read()
                
                # Extraer términos únicos del documento
                terms = set(content.split())
                
                # Para cada término único en el documento
                for term in terms:
                    # Obtener o asignar termID
                    term_id = self.term_id_map[term]
                    
                    # Si el término no está en postings, inicializar su lista
                    if term_id not in postings:
                        postings[term_id] = []
                    
                    # Añadir docID a la posting list del término
                    # Solo si no está ya (evitar duplicados, aunque con set no debería pasar)
                    if not postings[term_id] or postings[term_id][-1] != doc_id:
                        postings[term_id].append(doc_id)
        ### End your code

    def parse_all_subdirectories(self):
        """Parse all documents and generate dictionary of postings lists 
        Should use self.term_id_map and self.doc_id_map to get termIDs and docIDs.
        """
        assert not self.postings
        # Remember to sort the list of directories to assure they are read in the right order
        ### Begin your code
        # Obtener lista de subdirectorios y ordenarlos
        subdirs = sorted(os.listdir(self.data_dir))
        
        # Procesar cada subdirectorio en orden
        for subdir in subdirs:
            subdir_path = os.path.join(self.data_dir, subdir)
            
            # Verificar que es un directorio
            if os.path.isdir(subdir_path):
                print(f"Procesando subdirectorio: {subdir}")
                # Parsear todos los documentos del subdirectorio
                self.parse_directory(subdir_path, self.postings)
        
        print(f"Indexación completada: {len(self.term_id_map)} términos, {len(self.doc_id_map)} documentos")
        ### End your code

Utiliza los ficheros de juguete en la ruta indicada por `toy_dir` para probar tu código.

In [10]:
!find $toy_dir -type f | sort

data/toy/0/fine.txt
data/toy/0/hello.txt
data/toy/1/byebye.txt
data/toy/1/bye.txt
data/toy/2/fine.txt
data/toy/2/hello.txt


Comprueba si la función funciona como se espera con los datos de juguete.

In [11]:
with open(Path(toy_dir)/'0/fine.txt', 'r') as file:
    print(file.read())
with open(Path(toy_dir)/'0/hello.txt', 'r') as file:
    print(file.read())

i'm fine , thank you

hi hi
how are you ?



La siguiente celda comprueba si la función `parse_all_subdirectories` es capaz de escanear correctamente los ficheros de juguete situados en `toy_dir`, y a continuación, muestra el objeto índice creado con `obj.__dict__`, de forma que se pueda ver el valor del atributo `td_pairs`, entre otros.

Tras escanear los documentos y construir el diccionario sobre los datos de juguete, deberíamos tener un atributo `postings_lists` con el siguiente contenido:

In [12]:
toy_index = NaiveIndex(toy_dir, out_dir_naive_toy)
toy_index.parse_all_subdirectories()
toy_index.__dict__

Procesando subdirectorio: 0
Procesando subdirectorio: 1
Procesando subdirectorio: 2
Indexación completada: 14 términos, 6 documentos


{'term_id_map': <__main__.IdMap at 0x7215f8271600>,
 'doc_id_map': <__main__.IdMap at 0x7215f8270fd0>,
 'data_dir': 'data/toy',
 'output_dir': 'output/naive/toy_index',
 'postings': {0: [0, 4],
  1: [0, 1, 2, 4, 5],
  2: [0, 4],
  3: [0, 2, 4],
  4: [0, 4],
  5: [1, 5],
  6: [1, 5],
  7: [1, 2, 5],
  8: [1, 5],
  9: [2],
  10: [2],
  11: [2, 3],
  12: [2],
  13: [2]},
 'postings_metadata': {}}

In [13]:
toy_index.postings

{0: [0, 4],
 1: [0, 1, 2, 4, 5],
 2: [0, 4],
 3: [0, 2, 4],
 4: [0, 4],
 5: [1, 5],
 6: [1, 5],
 7: [1, 2, 5],
 8: [1, 5],
 9: [2],
 10: [2],
 11: [2, 3],
 12: [2],
 13: [2]}

Resultado: 

```
{0: [0, 4],
 1: [0, 4],
 2: [0, 2, 4],
 3: [0, 4],
 4: [0, 1, 2, 4, 5],
 5: [1, 5],
 6: [1, 5],
 7: [1, 5],
 8: [1, 2, 5],
 9: [2],
 10: [2],
 11: [2],
 12: [2, 3],
 13: [2]}
```

Escribe algunas pruebas para asegurarte de que efectivamente el método `parse_all_subdirectories` funciona como se espera en los datos de juguete (`toy_dir`). Por ejemplo, deberías asegurarte de que una palabra dada recibe el mismo id cada vez que aparece).

In [14]:
print("Term->TermIDs")
### Begin your code
# Mostrar todos los términos y sus IDs
for term in toy_index.term_id_map.id_to_str:
    term_id = toy_index.term_id_map[term]
    print(f"('{term}', {term_id})")

# Verificar que un término recibe siempre el mismo ID
print("\nVerificación de consistencia:")
test_term = "you"
id1 = toy_index.term_id_map[test_term]
id2 = toy_index.term_id_map[test_term]
print(f"'{test_term}' -> ID primera consulta: {id1}, ID segunda consulta: {id2}")
assert id1 == id2, f"El término '{test_term}' no recibe el mismo ID"
print(f"✓ El término '{test_term}' recibe consistentemente el ID {id1}")

# Verificar que IDs diferentes corresponden a términos diferentes
print("\nVerificación bidireccional:")
for i in range(min(3, len(toy_index.term_id_map))):
    term = toy_index.term_id_map[i]
    recovered_id = toy_index.term_id_map[term]
    print(f"ID {i} -> '{term}' -> ID {recovered_id}")
    assert i == recovered_id, f"Mapeo inconsistente para ID {i}"
print(f"✓ Mapeo bidireccional correcto")

### End your code
print("\nDoc->DocIDs")
### Begin your code
# Mostrar todos los documentos y sus IDs
for doc_path in toy_index.doc_id_map.id_to_str:
    doc_id = toy_index.doc_id_map[doc_path]
    print(f"('{doc_path}', {doc_id})")

# Verificar que un documento recibe siempre el mismo ID
print("\nVerificación de consistencia:")
test_doc = toy_index.doc_id_map[0]  # Obtener el primer documento
id1 = toy_index.doc_id_map[test_doc]
id2 = toy_index.doc_id_map[test_doc]
print(f"'{test_doc}' -> ID primera consulta: {id1}, ID segunda consulta: {id2}")
assert id1 == id2, f"El documento '{test_doc}' no recibe el mismo ID"
print(f"✓ El documento recibe consistentemente el ID {id1}")

# Verificar el orden de procesamiento (deben estar ordenados)
print("\nVerificación de orden de procesamiento:")
print(f"Total de documentos procesados: {len(toy_index.doc_id_map)}")
for i in range(len(toy_index.doc_id_map)):
    doc = toy_index.doc_id_map[i]
    print(f"  DocID {i}: {doc}")
print(f"✓ Documentos procesados en orden")

# Verificar algunas postings lists
print("\nVerificación de posting lists:")
if 4 in toy_index.postings:  # termID para "you"
    postings_you = toy_index.postings[4]
    print(f"Posting list para 'you' (termID=4): {postings_you}")
    print(f"  Aparece en {len(postings_you)} documentos")
    assert all(postings_you[i] <= postings_you[i+1] for i in range(len(postings_you)-1)), \
        "La posting list no está ordenada"
    print(f"✓ Posting list ordenada correctamente")

### End your code

Term->TermIDs
('i'm', 0)
('you', 1)
('fine', 2)
(',', 3)
('thank', 4)
('hi', 5)
('how', 6)
('?', 7)
('are', 8)
('to', 9)
('good', 10)
('bye', 11)
('later', 12)
('see', 13)

Verificación de consistencia:
'you' -> ID primera consulta: 1, ID segunda consulta: 1
✓ El término 'you' recibe consistentemente el ID 1

Verificación bidireccional:
ID 0 -> 'i'm' -> ID 0
ID 1 -> 'you' -> ID 1
ID 2 -> 'fine' -> ID 2
✓ Mapeo bidireccional correcto

Doc->DocIDs
('0/fine.txt', 0)
('0/hello.txt', 1)
('1/bye.txt', 2)
('1/byebye.txt', 3)
('2/fine.txt', 4)
('2/hello.txt', 5)

Verificación de consistencia:
'0/fine.txt' -> ID primera consulta: 0, ID segunda consulta: 0
✓ El documento recibe consistentemente el ID 0

Verificación de orden de procesamiento:
Total de documentos procesados: 6
  DocID 0: 0/fine.txt
  DocID 1: 0/hello.txt
  DocID 2: 1/bye.txt
  DocID 3: 1/byebye.txt
  DocID 4: 2/fine.txt
  DocID 5: 2/hello.txt
✓ Documentos procesados en orden

Verificación de posting lists:
Posting list para 'you'

Resultado: 
```
Term->TermIDs
("i'm", 0)
('fine', 1)
(',', 2)
('thank', 3)
('you', 4)
('hi', 5)
('how', 6)
('are', 7)
('?', 8)
('bye', 9)
('good', 10)
('to', 11)
('see', 12)
('later', 13)
Doc->DocIDs
('data/toy/0/fine.txt', 0)
('data/toy/0/hello.txt', 1)
('data/toy/1/byebye.txt', 2)
('data/toy/1/bye.txt', 3)
('data/toy/2/fine.txt', 4)
('data/toy/2/hello.txt', 5)
```

## Recuperación de información sobre el índice en memoria

Ahora, añade a la clase `NaiveIndex` los siguientes métodos:

- `get_term_postings_from_mem`: Debe devolver la lista de *postings* (*docIds*) de un determinado término dado como argumento.

- `docids_to_docnames`: A partir de una lista de *postings* dada como argumento, debe devolver una lista en la que el elemento i-ésimo será la ruta al fichero correspondiente añ *docId* i-esimo en la lista dada.

- `retrieve`: Debe devolver los nombres de los documentos en los que aparece un término dado como argumento.

In [15]:
from collections import defaultdict

class NaiveIndex(NaiveIndex):

    def get_term_postings_from_mem(self, term):
        postings = []
        assert self.postings
        ### Begin your code        
        # Obtener el termID del término
        if term in self.term_id_map.str_to_id:
            term_id = self.term_id_map[term]
            # Si el término existe en el índice, obtener su posting list
            if term_id in self.postings:
                postings = self.postings[term_id]
        ### End your code        
        return postings

    def docids_to_docnames(self, postings):
        doc_names = []
        ### Begin your code        
        # Convertir cada docID a su nombre de documento correspondiente
        for doc_id in postings:
            doc_name = self.doc_id_map[doc_id]
            doc_names.append(doc_name)
        ### End your code        
        return doc_names

    def retrieve(self, term):
        postings = self.get_term_postings_from_mem(term)
        return self.docids_to_docnames(self.get_term_postings_from_mem(term))

Comprueba que tus métodos funcionan. Recuerda que, tras añadir nuevos métodos a una clase, debes volver a construir un nuevo objeto de dicha clase para que contenga los nuevos métodos añadidos por celdas como la anterior:

In [16]:
toy_index = NaiveIndex(toy_dir, out_dir_naive_toy)
toy_index.parse_all_subdirectories()

Procesando subdirectorio: 0
Procesando subdirectorio: 1
Procesando subdirectorio: 2
Indexación completada: 14 términos, 6 documentos


In [17]:
toy_index.get_term_postings_from_mem("hi")

[1, 5]

Resultado: 
```
[1, 5]
```

In [18]:
toy_index.retrieve("you")

['0/fine.txt', '0/hello.txt', '1/bye.txt', '2/fine.txt', '2/hello.txt']

Resultado:

```
['data/toy/0/fine.txt',
 'data/toy/0/hello.txt',
 'data/toy/1/bye.txt',
 'data/toy/2/fine.txt',
 'data/toy/2/hello.txt']
```

## Guardar el índice invertido en disco

Una vez construido el índice, queremos guardarlo en disco para poder utilizarlo posteriormente con el fin de recuperar información del mismo sin necesidad de volver a construirlo. Esto también nos permitirá liberar la memoria ocupada por las *postings lists*, que hasta este punto se mantienen en memoria (atributo `postings`). Los pasos a seguir son los siguientes:

1. Guardar en un fichero el índice, es decir, las *postings list* de todos los términos del vocabulario, en orden creciente por *termId*.

1. Guardar en un fichero los *metadatos* necesarios para localizar los *postings* de un término en el fichero de índice anterior.



### Escribir los `postings` a un fichero

Debes programar los métodos `save_term_postings` y `save_postings` de la clase `NaiveIndex`, como se indica a continuación:

- `save_term_postings`: A partir de un fichero `f` previamente abierto con `with open(...) as f`, debe obtener el *offset* actual y a continuación escribir la lista de *postings* dada como argumento. Para escribir  las *postings lists* (listas de *docIDs*) en el disco, utilizaremos el paquete `struct` y su método `pack`; el módulo `struct` contiene funciones estáticas de codificación y decodificación, que permiten transformar una lista de enteros a un array de bytes, y viceversa. [Ver documentación aquí](https://docs.python.org/3/library/struct.html). En particular, se puede obtener una lista de enteros así: `struct.pack(f'{len(term_postings)}i', *term_postings)`

- `save_postings`: Debe crear un fichero en el directorio de salida del índice `output_dir`, con el nombre indicado por la variable `postings_index_filename`, en el que se guardará la *postings list* de cada término del vocabulario, en orden creciente por *termId*. Debe hacer uso de la función `save_term_postings` anterior. Conforme se van guardando las *posting lists* en el fichero, será necesario construir un nuevo diccionario (atributo `postings_metadata`) con la correspondencia entre cada término (*termId*) y los metadatos para localizar sus *postings* en el fichero del índice. Para cada término, los metadatos necesarios para recuperar posteriormente los postings son: el desplazamiento (*offset*) dentro del fichero de índice donde comienza, la longitud de la lista de *postings* y el tamaño en bytes de dicha lista.

#### Construir el atributo `postings_metadata` de la clase `NaiveIndex`
Se trata de un diccionario que relaciona los termIDs con una tripleta de metadatos que es útil para leer y escribir las *postings lists* en el archivo de índice a/desde el disco:

* `start_position_in_index_file`:posición (en bytes) de la *posting list* en el fichero índice.

* `number_of_postings_in_list`: número de entradas (docIDs) de lista.

* `length_in_bytes_of_postings_list`: longitud en bytes de la *postings list* codificada.

Se supone que este mapeo se mantiene en memoria. 

In [19]:
import struct

class NaiveIndex(NaiveIndex):
    def save_term_postings(self, file, term_postings):
        at_offset = -1  # offset where the postings are going to be stored (-1 is a non valid value)
        num_bytes = 0
        assert term_postings, "save_term_postings expets a non-empty postings list"
        ### Begin your code        
        # Obtener la posición actual en el archivo (offset)
        at_offset = file.tell()
        
        # Empaquetar la lista de postings como bytes
        # Formato: '{N}i' donde N es el número de enteros de 4 bytes
        packed_data = struct.pack(f'{len(term_postings)}i', *term_postings)
        
        # Escribir los bytes en el archivo
        file.write(packed_data)
        
        # Calcular el número de bytes escritos
        num_bytes = len(packed_data)
        ### End your code        
        assert at_offset >= 0 and num_bytes > 0, "save_term_postings not functional"
        return at_offset, num_bytes
        
    def save_postings(self):
        """
        Writes the inverted index in self.postings_list to a file
        """
        filename = postings_index_filename
        assert not self.postings_metadata, "Postings metadata must be empty before index is saved to file"
        ### Begin your code      
        # Crear la ruta completa al archivo de postings
        filepath = os.path.join(self.output_dir, filename)
        
        # Abrir el archivo en modo binario de escritura
        with open(filepath, 'wb') as f:
            # Recorrer todos los términos en orden (los diccionarios mantienen orden de inserción desde Python 3.7)
            for term_id, postings_list in self.postings.items():
                # Guardar la posting list y obtener offset y tamaño
                offset, num_bytes = self.save_term_postings(f, postings_list)
                
                # Guardar los metadatos: (offset, longitud_lista, tamaño_en_bytes)
                self.postings_metadata[term_id] = (offset, len(postings_list), num_bytes)
        ### End your code   
        assert self.postings_metadata, "Postings metadata not built after index saved to file"

    def release_postings_from_memory(self):
        """
        Releases the postings from memory, keeping the metadata to locate postings in file
        """
        self.postings = {}
        # NO vaciar postings_metadata - se necesita para acceder al índice en disco


A continuación, prueba tu código y muestra `postings_metadata`:

In [20]:
toy_index = NaiveIndex(toy_dir, out_dir_naive_toy)
toy_index.parse_all_subdirectories()
toy_index.save_postings()

Procesando subdirectorio: 0
Procesando subdirectorio: 1
Procesando subdirectorio: 2
Indexación completada: 14 términos, 6 documentos


In [21]:
toy_index.postings_metadata

{0: (0, 2, 8),
 1: (8, 5, 20),
 2: (28, 2, 8),
 3: (36, 3, 12),
 4: (48, 2, 8),
 5: (56, 2, 8),
 6: (64, 2, 8),
 7: (72, 3, 12),
 8: (84, 2, 8),
 9: (92, 1, 4),
 10: (96, 1, 4),
 11: (100, 2, 8),
 12: (108, 1, 4),
 13: (112, 1, 4)}

Resultado:
```
{0: (0, 2, 8),
 1: (8, 2, 8),
 2: (16, 3, 12),
 3: (28, 2, 8),
 4: (36, 5, 20),
 5: (56, 2, 8),
 6: (64, 2, 8),
 7: (72, 2, 8),
 8: (80, 3, 12),
 9: (92, 1, 4),
 10: (96, 1, 4),
 11: (100, 1, 4),
 12: (104, 2, 8),
 13: (112, 1, 4)}
```

Si tras guardar el índice en disco, volcamos con `hexdump` el contenido del fichero en el que hemos escrito los *postings*, deberíamos observar los *docIds* de cada *postings list* codificados, uno tras otro, codificados como un entero de 4 bytes (en *little endian*). NOTA: La primera columna mostrada por `hexdump` corresponde al offset del primer byte del fichero que aparece en esa línea (similar a como ocurría en el programa `okteta` que viste en la asignatura de "Fundamentos de Computadores"); el resto de columnas son el contenido del fichero.

In [22]:
!ls $out_dir_naive_toy
!hexdump -C $out_dir_naive_toy/$postings_index_filename

docs.dict  postings.index  postings_metadata.dict  terms.dict
00000000  00 00 00 00 04 00 00 00  00 00 00 00 01 00 00 00  |................|
00000010  02 00 00 00 04 00 00 00  05 00 00 00 00 00 00 00  |................|
00000020  04 00 00 00 00 00 00 00  02 00 00 00 04 00 00 00  |................|
00000030  00 00 00 00 04 00 00 00  01 00 00 00 05 00 00 00  |................|
00000040  01 00 00 00 05 00 00 00  01 00 00 00 02 00 00 00  |................|
00000050  05 00 00 00 01 00 00 00  05 00 00 00 02 00 00 00  |................|
00000060  02 00 00 00 02 00 00 00  03 00 00 00 02 00 00 00  |................|
00000070  02 00 00 00                                       |....|
00000074
00000000  00 00 00 00 04 00 00 00  00 00 00 00 01 00 00 00  |................|
00000010  02 00 00 00 04 00 00 00  05 00 00 00 00 00 00 00  |................|
00000020  04 00 00 00 00 00 00 00  02 00 00 00 04 00 00 00  |................|
00000030  00 00 00 00 04 00 00 00  01 00 00 00 05 00 00 00  |...........

Resultado: 

```
00000000  00 00 00 00 04 00 00 00  00 00 00 00 04 00 00 00  |................|
00000010  00 00 00 00 02 00 00 00  04 00 00 00 00 00 00 00  |................|
00000020  04 00 00 00 00 00 00 00  01 00 00 00 02 00 00 00  |................|
00000030  04 00 00 00 05 00 00 00  01 00 00 00 05 00 00 00  |................|
00000040  01 00 00 00 05 00 00 00  01 00 00 00 05 00 00 00  |................|
00000050  01 00 00 00 02 00 00 00  05 00 00 00 02 00 00 00  |................|
00000060  02 00 00 00 02 00 00 00  02 00 00 00 03 00 00 00  |................|
00000070  02 00 00 00                                       |....|
00000074
```

### Escribir los metadatos que referencian al fichero de *postings* (`postings_metadata`)

Debes programar el método `save_postings_metadata` de la clase `NaiveIndex`, que guarde en un fichero el contenido del diccionario `postings_metadata`. El nombre del fichero vendrá dado por la variable `postings_metadata_filename` y se creará igualmente en el directorio de salida; para cada término del vocabulario, se deberá escribir en orden creciente por *termId*, junto con el propio *termId*, la tripleta de metadatos anteriomente descrita *(offset, num_postings, num_bytes)*. NOTA: **No se debe asumir** que el diccionario `postings_metadata` está ordenado por *termId* (aunque en esta versión *naïve* en realidad sí lo está).

In [23]:
from collections import defaultdict

class NaiveIndex(NaiveIndex):

    def save_postings_metadata(self):
        """
        Escribe un diccionario de offsets en un archivo binario.
        """
        filename = postings_metadata_filename
        if not self.postings_metadata:
            print("'save_postings_metadata' called, but no metadata found")
            return
        ### Begin your code
        # The values key, offset, length and size_in_bytes will be written as a 4-byte-integer (struct.pack('i',key)
        
        # Crear la ruta completa al archivo de metadatos
        filepath = os.path.join(self.output_dir, filename)
        
        # Abrir el archivo en modo binario de escritura
        with open(filepath, 'wb') as f:
            # Ordenar por termID (clave del diccionario) para asegurar orden creciente
            sorted_term_ids = sorted(self.postings_metadata.keys())
            
            # Para cada termID en orden
            for term_id in sorted_term_ids:
                # Obtener la tripleta de metadatos (offset, num_postings, num_bytes)
                offset, num_postings, num_bytes = self.postings_metadata[term_id]
                
                # Empaquetar y escribir: termID, offset, num_postings, num_bytes
                # Todos como enteros de 4 bytes ('i' = signed int de 4 bytes)
                packed_data = struct.pack('iiii', term_id, offset, num_postings, num_bytes)
                f.write(packed_data)
            
        ### End your code

In [24]:
toy_index = NaiveIndex(toy_dir, out_dir_naive_toy)
toy_index.parse_all_subdirectories()
toy_index.save_postings()
toy_index.save_postings_metadata()

Procesando subdirectorio: 0
Procesando subdirectorio: 1
Procesando subdirectorio: 2
Indexación completada: 14 términos, 6 documentos


In [25]:
toy_index.postings_metadata

{0: (0, 2, 8),
 1: (8, 5, 20),
 2: (28, 2, 8),
 3: (36, 3, 12),
 4: (48, 2, 8),
 5: (56, 2, 8),
 6: (64, 2, 8),
 7: (72, 3, 12),
 8: (84, 2, 8),
 9: (92, 1, 4),
 10: (96, 1, 4),
 11: (100, 2, 8),
 12: (108, 1, 4),
 13: (112, 1, 4)}

In [26]:
!ls $out_dir_naive_toy
!hexdump -C $out_dir_naive_toy/$postings_metadata_filename

docs.dict  postings.index  postings_metadata.dict  terms.dict
00000000  00 00 00 00 00 00 00 00  02 00 00 00 08 00 00 00  |................|
00000010  01 00 00 00 08 00 00 00  05 00 00 00 14 00 00 00  |................|
00000020  02 00 00 00 1c 00 00 00  02 00 00 00 08 00 00 00  |................|
00000030  03 00 00 00 24 00 00 00  03 00 00 00 0c 00 00 00  |....$...........|
00000040  04 00 00 00 30 00 00 00  02 00 00 00 08 00 00 00  |....0...........|
00000050  05 00 00 00 38 00 00 00  02 00 00 00 08 00 00 00  |....8...........|
00000060  06 00 00 00 40 00 00 00  02 00 00 00 08 00 00 00  |....@...........|
00000070  07 00 00 00 48 00 00 00  03 00 00 00 0c 00 00 00  |....H...........|
00000080  08 00 00 00 54 00 00 00  02 00 00 00 08 00 00 00  |....T...........|
00000090  09 00 00 00 5c 00 00 00  01 00 00 00 04 00 00 00  |....\...........|
000000a0  0a 00 00 00 60 00 00 00  01 00 00 00 04 00 00 00  |....`...........|
000000b0  0b 00 00 00 64 00 00 00  02 00 00 00 08 00 00 00  |....d...

Resultado:
```
00000000  00 00 00 00 00 00 00 00  02 00 00 00 08 00 00 00  |................|
00000010  01 00 00 00 08 00 00 00  02 00 00 00 08 00 00 00  |................|
00000020  02 00 00 00 10 00 00 00  03 00 00 00 0c 00 00 00  |................|
00000030  03 00 00 00 1c 00 00 00  02 00 00 00 08 00 00 00  |................|
00000040  04 00 00 00 24 00 00 00  05 00 00 00 14 00 00 00  |....$...........|
00000050  05 00 00 00 38 00 00 00  02 00 00 00 08 00 00 00  |....8...........|
00000060  06 00 00 00 40 00 00 00  02 00 00 00 08 00 00 00  |....@...........|
00000070  07 00 00 00 48 00 00 00  02 00 00 00 08 00 00 00  |....H...........|
00000080  08 00 00 00 50 00 00 00  03 00 00 00 0c 00 00 00  |....P...........|
00000090  09 00 00 00 5c 00 00 00  01 00 00 00 04 00 00 00  |....\...........|
000000a0  0a 00 00 00 60 00 00 00  01 00 00 00 04 00 00 00  |....`...........|
000000b0  0b 00 00 00 64 00 00 00  01 00 00 00 04 00 00 00  |....d...........|
000000c0  0c 00 00 00 68 00 00 00  02 00 00 00 08 00 00 00  |....h...........|
000000d0  0d 00 00 00 70 00 00 00  01 00 00 00 04 00 00 00  |....p...........|
000000e0
```

### Guardar el índice al completo en disco

Ahora, estamos en disposición de crear un método `save` que se encargue de guardar el índice construido al completo en disco, en cuatro sencillos pasos:

1. Guardar el mapeo entre términos y *termId* usando el método `save_terms_dict`.

1. Guardar el mapeo entre rutas a documentos y *docIds* usando el método `save_docs_dict`.

1. Guardar el fichero de índice con los *postings*.

1. Guardar los metadatos para localizar los *postings* en el fichero de índice.

1. Liberar la memoria ocupada por los *postings* y sus metadatos, una vez que ambos han sido escritos a disco.

In [27]:
class NaiveIndex(NaiveIndex):
    def save_terms_dict(self):
        with open(os.path.join(self.output_dir, terms_filename), 'wb') as f:
            pkl.dump(self.term_id_map, f)
    def save_docs_dict(self):
        with open(os.path.join(self.output_dir, docs_filename), 'wb') as f:
            pkl.dump(self.doc_id_map, f)
    
    def save(self):
        """Dumps doc_id_map and term_id_map into output directory"""
        ### Begin your code        
        # 1. Guardar el mapeo término -> termID
        self.save_terms_dict()
        
        # 2. Guardar el mapeo documento -> docID
        self.save_docs_dict()
        
        # 3. Guardar el fichero de índice con los postings
        self.save_postings()
        
        # 4. Guardar los metadatos para localizar los postings
        self.save_postings_metadata()
        
        # 5. Liberar la memoria ocupada por postings y metadatos
        self.release_postings_from_memory()
        
        ### End your code

In [28]:
toy_index = NaiveIndex(toy_dir, out_dir_naive_toy)
toy_index.parse_all_subdirectories()
toy_index.save()

Procesando subdirectorio: 0
Procesando subdirectorio: 1
Procesando subdirectorio: 2
Indexación completada: 14 términos, 6 documentos


## Cargar los metadatos del índice en disco para poder recuperar información

A partir de este momento, el fichero de *postings* se mantiene en disco, y sólo es necesario mantener en memoria los metadatos necesarios para poder acceder a las partes del fichero de *postings* donde se guardan los *docIds* en los que aparecen los términos de la consulta. Así pues, para poder realizar recuperación de información sobre el índice en disco, debes programar los siguientes métodos de `NaiveIndex`:

- `load_next_metadata_entry`: Lee una entrada del fichero de metadatos, y devuelve una cuádrupla con el *termId* y los metadatos para leer la *posting list* del fichero de índice. La cuádrupla estará  formada por *(termId, offset, número de *postings*, tamaño en bytes de la lista)*.

- `load_postings_metadata`: Carga el fichero de metadatos de disco, haciendo uso de la función `load_next_metadata_entry` y construye el atributo `postings_metadata` de la clase `NaiveIndex`.

- `load`: Método que utilizará el código cliente que haga uso de `NaiveIndex` para cargar un índice y poder utilizarlo para realizar consultas. Cargará en memoria tanto el diccionario (mapeo de términos con *termIds* y documentos con *docIds*) como los metadatos.

In [29]:
class NaiveIndex(NaiveIndex):
    def load_terms_dict(self):
        ## Load term<->termID mapping
        with open(os.path.join(self.output_dir, terms_filename), 'rb') as f:
            self.term_id_map = pkl.load(f)
    def load_docs_dict(self):
        ## Load doc<->docID mapping
        with open(os.path.join(self.output_dir, docs_filename), 'rb') as f:
            self.doc_id_map = pkl.load(f)

    def load_next_metadata_entry(self, f):
        key_data = None
        key_data = f.read(4)
        if key_data:
            key = struct.unpack('i', key_data)[0]
            offset, length, size_in_bytes = struct.unpack('iii', f.read(12))
            return key, offset, length, size_in_bytes
        return None,None,None,None
        
    def load_postings_metadata(self):
        """ Load postings metadata from file """
        filename = postings_metadata_filename
        assert not self.postings_metadata
        ### Begin your code
        # Crear la ruta completa al archivo de metadatos
        filepath = os.path.join(self.output_dir, filename)
        
        # Abrir el archivo en modo binario de lectura
        with open(filepath, 'rb') as f:
            # Leer todas las entradas hasta el final del archivo
            while True:
                # Leer una entrada usando load_next_metadata_entry
                term_id, offset, length, size_in_bytes = self.load_next_metadata_entry(f)
                
                # Si no hay más entradas (key es None), salir del bucle
                if term_id is None:
                    break
                
                # Guardar los metadatos: termID -> (offset, length, size_in_bytes)
                self.postings_metadata[term_id] = (offset, length, size_in_bytes)
        
        ### End your code
           
    def load(self):
        """Loads index from output directory"""
        assert not self.postings_metadata
        ### Begin your code        
        # 1. Cargar el mapeo término <-> termID
        self.load_terms_dict()
        
        # 2. Cargar el mapeo documento <-> docID
        self.load_docs_dict()
        
        # 3. Cargar los metadatos para acceder a los postings en disco
        self.load_postings_metadata()
        
        ### End your code

## Recuperación de información a partir del índice en disco

Debes programar los siguientes métodos de la clase `NaiveIndex`, necesarios para leer de disco el diccionario y poder realizar recuperación de información a partir del índice invertido:

- `load_posting_at_file_offset`: Debes desplazarte al `offset` del fichero `f` que se pasa como parámetro y a partir de ahí leer `length` enteros de 4 bytes. Usa `struct.unpack` para extraer la lista de los enteros leída.

- `get_term_postings_from_file`: Debe asumir que los `postings_metadata` están ubicados en memoria, y a partir del mapeo de términos a *termId*, debe acceder al desplazamiento adecuado del fichero del índice y leer la *posting list* del término, haciendo uso de la función `load_next_posting` anterior.

- `retrieve`: Es la función central de recuperación de información mediante el índice. Debe comprobar si los *postings* están ubicados en memoria (atributo `postings`) o en disco, en cuyo caso usará `get_term_postings_from_file` para leer únicamente de disco los *postings* del término consultado. En caso de que el índice esté íntegramente cargado en memoria, evitará acceder a disco. Adicionalmente, recibirá un argumento `sanity_check`, en función del cual el método comprobará que `get_term_postings_from_file` devuelve lo mismo que `get_term_postings_from_mem`, siempre que tanto `postings` como `postings_metadata` estén adecuadamente construidos.

In [30]:
class NaiveIndex(NaiveIndex):

    def load_posting_at_file_offset(self, f, offset, length):
        assert length > 0
        postings_list = None
        ### Begin your code
        # Desplazarse al offset especificado en el archivo
        f.seek(offset)
        
        # Leer 'length' enteros de 4 bytes (length * 4 bytes en total)
        num_bytes = length * 4
        data = f.read(num_bytes)
        
        # Desempaquetar los datos como una lista de enteros
        # Formato: '{length}i' donde length es el número de enteros
        postings_list = list(struct.unpack(f'{length}i', data))
        
        ### End your code
        return postings_list

    def get_term_postings_from_file(self, term):
        disk_postings = []
        assert self.postings_metadata, "Must read dictionary from disk before retrieval!"
        ### Begin your code   
        # Verificar si el término existe en el mapeo de términos
        if term not in self.term_id_map.str_to_id:
            # El término no existe en el vocabulario, devolver lista vacía
            return disk_postings
        
        # Obtener el termID del término
        term_id = self.term_id_map[term]
        
        # Verificar si el término tiene metadatos (existe en el índice)
        if term_id not in self.postings_metadata:
            # El término no tiene postings, devolver lista vacía
            return disk_postings
        
        # Obtener los metadatos del término
        offset, length, size_in_bytes = self.postings_metadata[term_id]
        
        # Abrir el archivo de índice y leer la posting list
        filepath = os.path.join(self.output_dir, postings_index_filename)
        with open(filepath, 'rb') as f:
            disk_postings = self.load_posting_at_file_offset(f, offset, length)
        
        ### End your code
        return disk_postings

    def retrieve(self, term, sanity_check=False):
        postings = []
        sanity_postings = None
        where = None  # should be "memory" if the postings are in memory or "disk" if the postings are in disk
        ### Begin your code        
        # Comprobar si los postings están en memoria
        if self.postings:
            # Los postings están en memoria
            where = "memory"
            postings = self.get_term_postings_from_mem(term)
            
            # Si sanity_check está activado y hay metadatos, comparar con disco
            if sanity_check and self.postings_metadata:
                sanity_postings = self.get_term_postings_from_file(term)
                assert postings == sanity_postings, \
                    f"Sanity check failed: postings from memory {postings} != postings from disk {sanity_postings}"
                print(f"✓ Sanity check passed for term '{term}': memory and disk postings match")
        
        elif self.postings_metadata:
            # Los postings NO están en memoria, pero hay metadatos (índice en disco)
            where = "disk"
            postings = self.get_term_postings_from_file(term)
        
        else:
            # No hay ni postings en memoria ni metadatos
            raise RuntimeError("Index not loaded: no postings in memory and no metadata available")
        
        ### End your code
        doc_names = [self.doc_id_map[i] for i in postings]
        return doc_names

In [31]:
toy_index = NaiveIndex(toy_dir, out_dir_naive)
toy_index.parse_all_subdirectories()
toy_index.save()
toy_index.retrieve("you", sanity_check=True)

Procesando subdirectorio: 0
Procesando subdirectorio: 1
Procesando subdirectorio: 2
Indexación completada: 14 términos, 6 documentos


['0/fine.txt', '0/hello.txt', '1/bye.txt', '2/fine.txt', '2/hello.txt']

In [32]:
!grep -rlw "you" $toy_dir | sort

data/toy/0/fine.txt
data/toy/0/hello.txt
data/toy/1/bye.txt
data/toy/2/fine.txt
data/toy/2/hello.txt


## Indexando el corpus de documentos íntegro de forma *naïve* provoca `MemoryError`

Una vez hayas realizado un conjunto suficiente de pruebas sobre el conjunto de datos de juguete, es posible indexar el corpus al completo. Sin embargo, si estás aplicando los límites en el uso de memoria indicados, la celda siguiente mostrará un fallo `MemoryError` durante la ejecución, ya que generar el diccionario de términos con sus postings lists requiere más memoria de la disponible. 

> Si deseas completar el indexado usando el algoritmo *naive*, debes establecerlo en la variable `build_naive_index_full` al comienzo de este *notebook* y luego ejecutarlo con `jupyter-notebook` desde otro terminal del shell en el que **no hayas impuesto mediante `ulimit` ningún límite en el uso de la memoria**.

In [33]:
if build_naive_index_full:
    naive_index_full = NaiveIndex(corpus_dir, out_dir_naive)
    naive_index_full.parse_all_subdirectories()
    naive_index_full.save()

Procesando subdirectorio: 0
Procesando subdirectorio: 1
Procesando subdirectorio: 1
Procesando subdirectorio: 2
Procesando subdirectorio: 2
Procesando subdirectorio: 3
Procesando subdirectorio: 3
Procesando subdirectorio: 4
Procesando subdirectorio: 4
Procesando subdirectorio: 5
Procesando subdirectorio: 5
Procesando subdirectorio: 6
Procesando subdirectorio: 6
Procesando subdirectorio: 7
Procesando subdirectorio: 7
Procesando subdirectorio: 8
Procesando subdirectorio: 8
Procesando subdirectorio: 9
Procesando subdirectorio: 9
Indexación completada: 347071 términos, 98998 documentos
Indexación completada: 347071 términos, 98998 documentos


In [34]:
!ls -al $out_dir_naive

total 72996
drwxrwxr-x 2 pyros05 pyros05     4096 oct 19 18:33 .
drwxrwxr-x 4 pyros05 pyros05     4096 oct 19 18:25 ..
-rw-rw-r-- 1 pyros05 pyros05  6804541 oct 19 18:39 docs.dict
-rw-rw-r-- 1 pyros05 pyros05 55286272 oct 19 18:39 postings.index
-rw-rw-r-- 1 pyros05 pyros05  5553136 oct 19 18:39 postings_metadata.dict
-rw-rw-r-- 1 pyros05 pyros05  7088518 oct 19 18:39 terms.dict


Ahora probemos a usar el índice para recuperar todos los documentos que contienen el término "porcupine" (puercoespín).

In [35]:
if build_naive_index_full:
    naive_index_full.retrieve("porcupine", sanity_check=True)

In [36]:
!grep -rlw "porcupine" $corpus_dir | sort

data/corpus/5/searchworks.stanford.edu_view_5614345
data/corpus/6/wnt.stanford.edu_
data/corpus/7/www.scs.stanford.edu_07wi-cs244b_notes_
data/corpus/9/www.stanford.edu_group_dahlia_genetics_cultivars_valley_porcupine_valley_porcupine.htm
data/corpus/9/www.stanford.edu_group_nusselab_cgi-bin_wnt_inhibitors
data/corpus/9/www.stanford.edu_group_nusselab_cgi-bin_wnt_porcupine
data/corpus/9/www.stanford.edu_group_nusselab_cgi-bin_wnt_smallmolecules
data/corpus/9/www.stanford.edu_group_Urchin_language.htm


## Cargar índice desde disco

In [37]:
if build_naive_index_full:
    index_in_disk = NaiveIndex(corpus_dir, out_dir_naive)
    index_in_disk.load()
    index_in_disk.retrieve("porcupine")
    index_in_disk.retrieve("porcupine", sanity_check=True)